In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

crop_x, crop_y, crop_width, crop_height = 450, 150, 800, 800

#change count
save_frames = False
frame_count = 0
max_frames = 100

output_dir = "saved_frames"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cropped_frame = frame[crop_y:crop_y + crop_height, crop_x:crop_x + crop_width]

    original_frame = cropped_frame.copy()
    frame_rgb = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    annotated_frame = cropped_frame.copy()
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(annotated_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    combined_frame = np.hstack((original_frame, annotated_frame))

   
    cv2.imshow("Original and Annotated Preview", combined_frame)


    if save_frames:
        frame_filename = os.path.join(output_dir, f"hand{frame_count:04d}.png")
        cv2.imwrite(frame_filename, combined_frame)  
        frame_count += 1
        if frame_count >= max_frames:
            save_frames = False 
            print("Process complete")

    key = cv2.waitKey(1) & 0xFF
    if key == ord(' '):  
        save_frames = True
        frame_count = 0
        print("Started saving 100 frames...")
    elif key == 27:  
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import time
import csv

# Initialize Mediapipe Hand solutions
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Open the webcam
cap = cv2.VideoCapture(0)

# Map landmark indices to custom names
point_names = {
    0: "T0", 1: "T1", 2: "T2", 3: "T3", 4: "Thumb",
    5: "I3", 6: "I2", 7: "I1", 8: "Index",
    9: "M3", 10: "M2", 11: "M1", 12: "Middle",
    13: "R3", 14: "R2", 15: "R1", 16: "Ring",
    17: "L3", 18: "L2", 19: "L1", 20: "Little"
}

# Initialize a timer, CSV file, and frame counter
last_time = time.time()
csv_filename = "hand_coordinates_with_gesture.csv"
frame_counter = 1  # Frame counter for image naming

# Write header to CSV if it doesn't exist
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Gesture", "Timestamp", "Image Name"] + [point_names[i] for i in range(21)])

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty frame.")
        continue

    # Flip the frame horizontally and convert the BGR image to RGB
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        current_time = time.time()
        # Check if 0.5 seconds have passed
        if current_time - last_time >= 0.5:
            last_time = current_time
            for hand_landmarks in results.multi_hand_landmarks:
                all_points_coordinates = []
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    # Get normalized coordinates
                    x = landmark.x * frame.shape[1]
                    y = landmark.y * frame.shape[0]
                    all_points_coordinates.append((int(x), int(y)))

                    # Draw a circle at each landmark
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)

                    # Add labels with custom names
                    cv2.putText(frame, f"{point_names[idx]}", (int(x), int(y) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # Save the frame as an image
                image_name = f"{frame_counter}.png"
                cv2.imwrite(image_name, frame)

                # Prepare row data with the gesture, timestamp, and image name
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(current_time))
                row_data = ["courser", timestamp, image_name] + [
                    f"{coord[0]},{coord[1]}" for coord in all_points_coordinates
                ]

                # Write the row to the CSV file
                with open(csv_filename, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(row_data)

                # Increment the frame counter
                frame_counter += 1

                # Print the collected data
                print("Gesture, Timestamp, Image Name, and All Points' Coordinates:", row_data)

                # Draw hand landmarks on the frame
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the frame
    cv2.imshow("Hand Tracking with Custom Point Names and Gesture Column", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


Gesture and All Points' Coordinates: ['courser', '436,351', '418,341', '407,319', '411,300', '420,290', '413,297', '414,270', '416,253', '416,239', '428,297', '431,278', '431,289', '430,302', '444,302', '446,284', '446,294', '444,305', '458,309', '461,292', '458,300', '456,309']
Gesture and All Points' Coordinates: ['courser', '473,267', '456,258', '442,238', '445,217', '456,203', '443,212', '439,184', '439,167', '441,152', '458,214', '461,196', '459,217', '458,230', '473,218', '474,202', '469,223', '467,236', '486,224', '485,214', '480,228', '477,237']
Gesture and All Points' Coordinates: ['courser', '461,273', '444,263', '429,243', '434,223', '449,213', '434,212', '432,185', '432,169', '434,154', '449,215', '450,199', '448,219', '446,232', '463,220', '464,207', '458,228', '455,240', '476,228', '475,219', '468,234', '464,243']
Gesture and All Points' Coordinates: ['courser', '402,261', '388,258', '372,241', '371,222', '382,211', '379,210', '375,187', '372,170', '371,157', '394,209', '